In [1]:
!pip install --upgrade tables
!pip install eli5
!pip install xgboost


     |████████████████████████████████| 4.3MB 3.2MB/s 
  Found existing installation: tables 3.4.4
    Uninstalling tables-3.4.4:
      Successfully uninstalled tables-3.4.4
     |████████████████████████████████| 112kB 3.5MB/s 


In [3]:
cd 'drive/My Drive/Colab Notebooks/challenge-matrix/Challenge 2/'

/content/drive/My Drive/Colab Notebooks/challenge-matrix/Challenge 2


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.dummy import DummyRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor


import xgboost as xgb

from sklearn.metrics import mean_absolute_error as mea
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)
Using TensorFlow backend.


In [0]:
df = pd.read_hdf('data/car.h5')

In [0]:
df = df [ df['price_currency'] != 'EUR'] 

In [0]:
Suff_Cat = '__cat'
for feat in df.columns:
  if isinstance(df[feat][0], list): continue
  fac_val = df[feat].factorize()[0]

  if (Suff_Cat in feat):
    df[feat] = fac_val
  else:
    df [feat + Suff_Cat] = fac_val

In [0]:
cat_feats = [x for x in df.columns if Suff_Cat in x]

In [0]:
cat_feats = [x for x in cat_feats if 'price' not in x]

In [0]:
X = df[cat_feats].values
y = df ['price_value'].values

model = DecisionTreeRegressor (max_depth=5)

scores = cross_val_score(model, X, y, cv=3,scoring="neg_mean_absolute_error")

In [12]:
np.mean(scores)

-19566.588937368328

In [13]:
n = DecisionTreeRegressor(max_depth=5)
n.fit(X, y)

imp = PermutationImportance(n, random_state=0).fit(X, y)
eli5.show_weights(imp, feature_names=cat_feats)

Weight,Feature
0.2533 ± 0.0040,param_napęd__cat
0.2008 ± 0.0047,param_faktura-vat__cat
0.1943 ± 0.0088,param_stan__cat
0.1423 ± 0.0086,param_rok-produkcji__cat
0.0629 ± 0.0047,param_moc__cat
0.0424 ± 0.0014,feature_kamera-cofania__cat
0.0412 ± 0.0008,param_skrzynia-biegów__cat
0.0286 ± 0.0037,param_marka-pojazdu__cat
0.0191 ± 0.0022,param_pojemność-skokowa__cat
0.0163 ± 0.0005,feature_bluetooth__cat


In [0]:
def run_model(model, feats):
  X = df[feats].values
  y = df ['price_value'].values


  scores = cross_val_score(model, X, y, cv=3,scoring="neg_mean_absolute_error")
  return np.mean(scores), np.std(scores)

In [15]:
model = DecisionTreeRegressor (max_depth=5)

run_model(model, cat_feats)

(-19566.588937368328, 90.61814865166)

In [16]:
run_model(RandomForestRegressor(n_estimators=50, random_state=0, max_depth=5), cat_feats)

(-18734.2072708522, 109.87074106274046)

In [0]:
param = {
    'max_depth':5,
    'n_estimators':50,
    'learning_rate':0.1,
    'seed':0

}

In [18]:
run_model(xgb.XGBRegressor(**param) , cat_feats)

[19:40:11] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:40:30] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:40:49] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-13039.290196724838, 109.36715375706265)

In [19]:
n = xgb.XGBRegressor(**param)
n.fit(X, y)

imp = PermutationImportance(n, random_state=0).fit(X, y)
eli5.show_weights(imp, feature_names=cat_feats)

[19:42:41] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


Weight,Feature
0.1209 ± 0.0019,param_napęd__cat
0.1175 ± 0.0030,param_rok-produkcji__cat
0.1113 ± 0.0013,param_stan__cat
0.0625 ± 0.0019,param_skrzynia-biegów__cat
0.0527 ± 0.0016,param_faktura-vat__cat
0.0461 ± 0.0015,param_moc__cat
0.0275 ± 0.0008,param_marka-pojazdu__cat
0.0230 ± 0.0004,param_typ__cat
0.0227 ± 0.0007,feature_kamera-cofania__cat
0.0191 ± 0.0007,param_pojemność-skokowa__cat


In [0]:
feats = 'param_napęd__cat','param_faktura-vat__cat','param_stan__cat','param_rok-produkcji__cat','param_moc__cat','feature_kamera-cofania__cat','param_skrzynia-biegów__cat','param_marka-pojazdu__cat','param_pojemność-skokowa__cat','feature_bluetooth__cat','feature_łopatki-zmiany-biegów__cat','feature_światła-led__cat','feature_klimatyzacja-manualna__cat','param_kod-silnika__cat',

In [0]:
feats = ['param_napęd__cat','param_rok-produkcji__cat','param_stan__cat','param_skrzynia-biegów__cat','param_faktura-vat__cat','param_moc__cat','param_marka-pojazdu__cat','param_typ__cat','feature_kamera-cofania__cat','param_pojemność-skokowa__cat','seller_name__cat','param_kod-silnika__cat','param_model-pojazdu__cat','feature_wspomaganie-kierownicy__cat','param_wersja__cat']

In [0]:
param = {
    'max_depth':5,
    'n_estimators':10,
    'learning_rate':0.1,
    'seed':0

}

In [22]:
run_model(xgb.XGBRegressor(**param) , feats)

[19:51:49] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:51:50] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:51:51] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-21082.741764181057, 196.74361679634063)

In [0]:
df['param_rok-produkcji'] = df['param_rok-produkcji'].map(lambda x: -1 if str(x) == 'None' else int(x))


In [0]:
feats = ['param_napęd__cat','param_rok-produkcji','param_stan__cat','param_skrzynia-biegów__cat','param_faktura-vat__cat','param_moc__cat','param_marka-pojazdu__cat','param_typ__cat','feature_kamera-cofania__cat','param_pojemność-skokowa__cat','seller_name__cat','param_kod-silnika__cat','param_model-pojazdu__cat','feature_wspomaganie-kierownicy__cat','param_wersja__cat']

In [25]:
run_model(xgb.XGBRegressor(**param) , feats)

[19:55:44] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:55:45] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:55:45] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-19092.455526517777, 88.67256304918803)

In [0]:
df['param_moc'] = df['param_moc'].map(lambda x: -1 if str(x) == 'None' else x.split(' ')[0])

In [0]:
feats = ['param_napęd__cat','param_rok-produkcji','param_stan__cat','param_skrzynia-biegów__cat','param_faktura-vat__cat','param_moc','param_marka-pojazdu__cat','param_typ__cat','feature_kamera-cofania__cat','param_pojemność-skokowa__cat','seller_name__cat','param_kod-silnika__cat','param_model-pojazdu__cat','feature_wspomaganie-kierownicy__cat','param_wersja__cat']

In [0]:
param = {
    'max_depth':5,
    'n_estimators':50,
    'learning_rate':0.1,
    'seed':0

}

In [30]:
run_model(xgb.XGBRegressor(**param) , feats)

[19:59:33] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:59:37] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[19:59:40] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-9676.350215790451, 57.870465142897594)

In [0]:
df['param_pojemność-skokowa'] = df['param_pojemność-skokowa'].map(lambda x: -1 if str(x) == 'None' else int(x.split('cm')[0].replace(' ','')))

In [0]:
feats = ['param_napęd__cat','param_rok-produkcji','param_stan__cat','param_skrzynia-biegów__cat','param_faktura-vat__cat','param_moc','param_marka-pojazdu__cat','param_typ__cat','feature_kamera-cofania__cat','param_pojemność-skokowa','seller_name__cat','param_kod-silnika__cat','param_model-pojazdu__cat','feature_wspomaganie-kierownicy__cat','param_wersja__cat']

In [33]:
run_model(xgb.XGBRegressor(**param) , feats)

[20:02:21] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:02:24] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:02:28] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


(-9506.071607840146, 72.38799990395903)